In [ ]:
import pandas as pd
import load_data
import importlib
import json
import math

import llama_cpp
from llama_cpp import Llama

# empty gpu
import gc
import torch
import os

In [ ]:
questions, context = load_data.get_siqa_data()

In [4]:
torch.cuda.empty_cache() 
gc.collect()

0

In [5]:
# !wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q6_K.gguf

In [ ]:
!gpustat -cp

In [9]:
# model_path = "BIG-bench/llama-2-13b.Q4_0.gguf"
model_path = "BIG-bench/llama-2-13b-chat.Q6_K.gguf"

In [3]:
context_dict = {'context': context[:1000]}  
context_df = pd.DataFrame(context_dict) 
    
# write the DataFrame to a CSV file
# df.to_csv('SIQA_context_1000.csv', index=True, index_label='ID') 

In [ ]:
llm = Llama(model_path=model_path, n_gpu_layers=-1, n_ctx=4096, logits_all=True, verbose=False)

In [49]:
def get_perplexity(prompt):
    # t = " " + target[j]

    # choice = bytes(t, encoding='utf-8')

    num_t = llm.tokenize(text=bytes(prompt,encoding='utf-8'))
    # print(num_t, len(num_t))
    num_t = len(num_t) - 1
    
    result = llm(prompt, echo=True, logprobs=1, presence_penalty=0, max_tokens=1)
    # print(result)

    # print(result['choices'][0]['logprobs']['tokens'][-tokens_choice_len:-1], result['choices'][0]['logprobs']['token_logprobs'][-tokens_choice_len:-1])
    # print(len(result['choices'][0]['logprobs']['token_logprobs'][0:-1]),num_t)
    seq_score = -sum(result['choices'][0]['logprobs']['token_logprobs'][1:-1])/num_t
    
    perplexity = math.exp(
    
    return seq_score

In [50]:
task_module_name = f"BIG-bench.bigbench.benchmark_tasks.social_iqa"
task_module = importlib.import_module(task_module_name)
module_path = list(task_module.__path__)[0]
json_path = os.path.join(module_path, "task.json")

In [ ]:
contexts_all = []
perplexity_all = []
with open(json_path, 'r') as f:
    dataset = json.load(f)['examples']
    
for sample in dataset:
    inp = sample['input']
    c = inp.split(".")[0].strip() # get context string
    llm.reset()
    print("---------------")
    print("Sentence: ", c,"\n")
    perplexity = get_perplexity(c)
    print("Perplexity: ", perplexity)
    contexts_all.append(c)
    perplexity_all.append(perplexity)


In [ ]:
import pandas as pd

In [ ]:
perplexity_dict = {'context': contexts_all, 'perplexity': perplexity_all} 
    
df = pd.DataFrame(perplexity_dict)

In [ ]:
df.to_csv("logs/"+"baseline_perplexity.csv")